### 1. C++拓展pytorch 的CPU

In [4]:
#以字符串的形式写入文件，建立test.cpp文件
head_file =r'''
#include <torch/extension.h> 
#include <torch/torch.h>

#include <vector>
#include <iostream>

torch::Tensor functest(torch::Tensor a,torch::Tensor b)
{
    return torch::add(a,b);
}

class Name{
public:
    Name(int a,int b){
    a_value =a;
    b_value =b;
    }
    int get_a(){return a_value;}
    int get_b(){return b_value;}
private:
    int a_value;
    int b_value;
};


//TORCH+EXTENSIO+NAME是模型的名字，不是字符串，
//这个量一定要查看编译命令，看看有没有定义
PYBIND11_MODULE(TORCH_EXTENSION_NAME myadd, m) {
    //python中名字   函数指针    函数注释
  m.def("forward", &functest, "new forward");
  
  //类的引入
  pybind11::class_<Name>(m,"pyname")
  .def(pybind11::init<int,int>())
  .def("get_a",&Name::get_a)
  .def("get_b",&Name::get_b);
}
'''
with open('test.cpp','w') as f:
    f.write(head_file)

### 第一种setuptools工具

#### setup.py文件如下

In [6]:
#把注释删除
head_file = '''
from setuptools import setup, Extension
from torch.utils import cpp_extension
import os
include_dirs = os.path.dirname(os.path.abspath(__file__))
setup(name='myop_file',#这个是文件名字，不是模块的名字,导入是下面的名字
      ext_modules=[cpp_extension.CppExtension('myadd',#这个名字是模型名字，要设置好
                                              ['test.cpp'],
                                              include_dirs=[include_dirs])],#这个里面给定的才是模块名称
      cmdclass={'build_ext': cpp_extension.BuildExtension})
'''
with open('setup.py','w') as f:
    f.write(head_file)

In [ ]:
#### 编译文件,生成模块包
!python setup.py install

In [ ]:
import torch #一定要先导入这个，因为构建的op.so库文件在/root/.cache/torch_extensions下面
#一定要进入build/ 下面的文件lib.linux-x86_64-3.7/才能导入这个包
#至关重要！！！！！！！！一定要进入build.lib（找到myadd的包）路径下面才能导入包
#myadd.
import myadd
a =torch.rand(2,2)
b=torch.rand(2,2)
myadd.forward(a,b)

### 第二种 

In [ ]:
from torch.utils.cpp_extension import load

modelfunc = load(name="model", #这个是模块名字，后面要import 这个
                sources=["test.cpp"],
               verbose=True, #显示编译过程
               )
#可以这样直接使用
modelfunc.forward(10,20)
#也可以这样用
import torch
import model
model.forward(12,10)

## load_line

In [ ]:
import torch
from torch.utils.cpp_extension import load_inline

In [ ]:
source = '''
int  add(int  x, int  y) {
  return x+ y;
}
'''
module = torch.utils.cpp_extension.load_inline(name='test',#生成在/root/.cache/torch_extensions
                                               cpp_sources=[source],#放在列表中
                                               functions=['add'])#函数指针，要和C++中的一样

module.add(10,20)